The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 複数の組み合わせの分析
配色(色の組み合わせ)を分析していく。まずは既知の配色パターンに分類してみる。

1. モノクロマティック配色（単色配色）
1つの色相（色の種類）をベースに、明度や彩度を変化させて作る配色です。統一感があり、シンプルで落ち着いた印象を与えます。

2. アナロゴス配色（類似色配色）
色相環で隣り合う2〜3色を組み合わせる手法です。調和が取りやすく、自然でまとまりのある印象を与えるため、ナチュラルなデザインに適しています。

3. コンプリメンタリー配色（補色配色）
色相環で真逆に位置する2色を組み合わせる手法です。例えば、赤と緑、青とオレンジなどです。対照的な色の組み合わせにより、視覚的に強いコントラストが生まれ、インパクトのあるデザインが可能です。

4. スプリットコンプリメンタリー配色（分割補色配色）
補色配色の変形で、基準となる色と、補色の隣接する2色を組み合わせます。補色配色ほどコントラストは強くないものの、調和と適度なコントラストを両立させます。

5. トライアド配色（三色配色）
色相環上で等距離に位置する3色を組み合わせる手法です。例えば、赤・青・黄のような組み合わせです。色のバランスが良く、カラフルで動きのあるデザインが可能です。

6. テトラード配色（四角形配色）
色相環上で正方形または長方形を描くように4色を選ぶ配色です。2組の補色関係を持つ色の組み合わせで、複雑かつ華やかな印象を与えます。

7. ナチュラル配色
自然界に見られる配色を参考にした手法で、落ち着いた、または柔らかな印象を与えます。森林や空、海などの自然の景観を元に配色が考えられることが多いです。

8. ニュートラル配色
グレー、ベージュ、白、黒など、彩度が低い色を中心にした配色です。シンプルで上品な印象を与えたい場合や、フォーマルな場面でよく使われます。

In [15]:
%load_ext autoreload
%autoreload 1
%aimport scripts.color3d

import json
import scripts.color as color
import numpy as np
import itertools as itertools 

# 色相の距離を計算する関数
def calculate_hue_distance(hue1, hue2):
    """2つの色相間の距離を計算する"""
    diff = abs(hue1 - hue2)
    return min(diff, 360 - diff)

# 各配色手法のスコアリング関数
def score_monochromatic(hsl_palette):
    """モノクロマティック配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    max_diff = max(calculate_hue_distance(hues[i], hues[j]) for i in range(len(hues)) for j in range(i+1, len(hues)))
    if max_diff <= 5:  # 5度以内ならモノクロマティック
        return 5
    return 0

def score_analogous(hsl_palette):
    """アナロゴス配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    max_diff = max(calculate_hue_distance(hues[i], hues[j]) for i in range(len(hues)) for j in range(i+1, len(hues)))
    if max_diff <= 30:  # 30度以内ならアナロゴス
        return 5
    elif max_diff <= 60:
        return 3
    return 0

def score_complementary(hsl_palette):
    """補色配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    if any(calculate_hue_distance(hues[i], hues[j]) in range(175, 185) for i in range(len(hues)) for j in range(i+1, len(hues))):
        return 5
    return 0

def score_split_complementary(hsl_palette):
    """スプリット補色配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    for i in range(len(hues)):
        for j in range(i+1, len(hues)):
            diff = calculate_hue_distance(hues[i], hues[j])
            if 150 <= diff <= 180:
                return 5
    return 0

def score_triad(hsl_palette):
    """トライアド配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    for i in range(len(hues)):
        for j in range(i+1, len(hues)):
            diff = calculate_hue_distance(hues[i], hues[j])
            if 115 <= diff <= 125:
                return 5
    return 0

def score_tetrad(hsl_palette):
    """テトラード配色のスコアを計算する"""
    hues = [hsl.h for hsl in hsl_palette]
    for i in range(len(hues)):
        for j in range(i+1, len(hues)):
            diff = calculate_hue_distance(hues[i], hues[j])
            if 85 <= diff <= 95:
                return 5
    return 0

def score_oklch_balance(oklch_palette):
    """OKLCHに基づく視覚的調和のスコアを計算する"""
    distances = [np.linalg.norm([oklch_palette[i].L - oklch_palette[j].L,
                                 oklch_palette[i].C - oklch_palette[j].C,
                                 oklch_palette[i].H - oklch_palette[j].H])
                 for i in range(len(oklch_palette)) for j in range(i+1, len(oklch_palette))]
    avg_distance = sum(distances) / len(distances)
    if avg_distance < 0.2:  # 小さいほど調和が取れている
        return 5
    return 0

# 総合スコアを計算する関数
def calculate_total_score(hex_palette):
    # Hex色をsRGB、HSL、OKLCHに変換
    rgb_palette = [hex_color.to_srgb() for hex_color in hex_palette]
    hsl_palette = [hex_color.to_hsl() for hex_color in hex_palette]
    oklch_palette = [hex_color.to_oklch() for hex_color in hex_palette]
    
    scores = {
        "monochromatic": score_monochromatic(hsl_palette),
        "analogous": score_analogous(hsl_palette),
        "complementary": score_complementary(hsl_palette),
        "split_complementary": score_split_complementary(hsl_palette),
        "triad": score_triad(hsl_palette),
        "tetrad": score_tetrad(hsl_palette),
        "oklch_balance": score_oklch_balance(oklch_palette)
    }
    
    # 最もスコアが高い配色手法を選択
    best_match = max(scores, key=scores.get)
    
    return best_match, scores

# 3色ずつの組み合わせで判定
def determine_color_scheme_for_4_colors(hex_palette):
    # 3色の組み合わせを抽出
    color_combinations = list(itertools.combinations(hex_palette, 3))
    combination_scores = []

    # 各3色の組み合わせに対して配色手法を判定
    for combination in color_combinations:
        rgb_palette = [hex_color.to_srgb() for hex_color in combination]
        best_match, scores = calculate_total_score(rgb_palette)
        combination_scores.append((combination, best_match, scores))

    # 結果を統合して最終的な配色パターンを決定
    pattern_counts = {}
    for _, best_match, _ in combination_scores:
        if best_match in pattern_counts:
            pattern_counts[best_match] += 1
        else:
            pattern_counts[best_match] = 1

    # 最も多く出現したパターンを最終的な配色パターンとして決定
    final_pattern = max(pattern_counts, key=pattern_counts.get)

    return final_pattern, combination_scores

def oklch_palette_to_hex_palette(oklch_palette):
    """
    OKLCH色空間のパレットをHexに変換する関数。
    
    Parameters:
    oklch_palette (np.array): OKLCH色空間のカラーパレットデータ
    
    Returns:
    hex_palette (list): Hex色空間に変換されたカラーパレット
    """
    hex_palette = []
    for oklch_color in oklch_palette:
        oklch = color.Oklch(*oklch_color)
        srgb_color = oklch.to_srgb()  # OKLCHからsRGBへ変換
        hex_color = srgb_color.to_hex()  # sRGBからHexへ変換
        hex_palette.append(hex_color)
    return hex_palette

def process_color_data(data):
    """
    カラーパレットデータを処理し、各パレットに対して配色パターンを判定する。
    
    Parameters:
    data (np.array): OKLCH色空間で表されたカラーパレットデータ
    
    Returns:
    results (list): 各パレットに対して判定された配色パターンとその詳細
    """
    results = []
    
    for oklch_palette in data:
        # OKLCHパレットをHexに変換
        hex_palette = oklch_palette_to_hex_palette(oklch_palette)
        
        # 配色パターンの判定
        final_pattern, combination_scores = determine_color_scheme_for_4_colors(hex_palette)
        
        # 結果を保存
        result = {
            "hex_palette": [hex_color.hex_value for hex_color in hex_palette],
            "final_pattern": final_pattern,
            "combination_scores": combination_scores
        }
        results.append(result)
    
    return results

def save_results(results, output_path):
    """
    判定結果をJSONファイルとして保存する関数。
    
    Parameters:
    results (list): 判定された結果のリスト
    output_path (str): 結果を保存するファイルのパス
    """
    with open(output_path, 'w') as f:
        json.dump(results, f, indent=4)

def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return np.array(data)  # NumPy配列に変換

if __name__ == "__main__":
    # データの読み込み
    data_path = '../data/processed/train_oklchPalette.json'  # 訓練データのファイルパス
    data = load_data(data_path)
    
    # カラーパレットデータを処理し、配色パターンを判定
    results = process_color_data(data)
    
    # 結果の保存
    output_path = '../data/results/color_scheme_results.json'
    save_results(results, output_path)
    
    print(f"結果が {output_path} に保存されました。")

AttributeError: 'Srgb' object has no attribute 'to_srgb'